# Laboratorium 5 - rekomendacje grafowe

## Przygotowanie

 * dataset i potrzebne biblioteki są dokładnie takie same jak na poprzednim laboratorium
 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn gensim==3.8.3`

In [1]:
pip install numpy pandas sklearn gensim==3.8.3

Note: you may need to restart the kernel to use updated packages.


## Część 1. - przygotowanie danych

In [2]:
# importujemy wszystkie potrzebne pakiety

import math
import random
import numpy as np
import pandas

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split, KFold

In [3]:
SCORE_THRESHOLD = 4.0 # recenzje z co najmniej taka ocena wezmiemy pod uwage
VECTOR_SIZE = 20 # jak dlugie powinny byc wektory osadzen wierzcholkow
NEIGHBOURS_WINDOW = 11 # tylu sasiadow wezmiemy pod uwage w algorytmie Word2Vec (symetrycznie i wliczajac biezacy element)
PATH_LENGTH = 5 # dlugosc pojedynczej losowej sciezki
PATHS_COUNT_PER_NODE = 20 # liczba losowych sciezek zaczynajacych sie w kazdym z wierzcholkow

In [4]:
# wczytujemy oceny uytkownikow

ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
ratings = ratings.where(ratings['rating'] >= SCORE_THRESHOLD).dropna()
# rozszerzamy ID tak, by sie nie powtarzaly
ratings['userId'] = ratings['userId'].apply(lambda x: 'u_' + str(int(x)))
ratings['movieId'] = ratings['movieId'].apply(lambda x: 'm_' + str(int(x)))
ratings

,userId,movieId,rating
0,u_1,m_1,4.0
1,u_1,m_3,4.0
2,u_1,m_6,4.0
3,u_1,m_47,5.0
4,u_1,m_50,5.0
...,...,...,...
100830,u_610,m_166528,4.0
100831,u_610,m_166534,4.0
100832,u_610,m_168248,5.0
100833,u_610,m_168250,5.0


In [5]:
# wczytujemy gatunki filmow

movies = pandas.read_csv('ml-latest-small/movies.csv').drop(columns=['title'])
movies['movieId'] = movies['movieId'].apply(lambda x: 'm_' + str(int(x)))
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
movies_to_genres = movies.explode('genres')
movies_to_genres['genres'] = movies_to_genres['genres'].apply(lambda x: 'g_' + x.lower())
movies_to_genres = movies_to_genres.rename(columns = {'genres': 'genre'})
movies_to_genres

,movieId,genre
0,m_1,g_adventure
0,m_1,g_animation
0,m_1,g_children
0,m_1,g_comedy
0,m_1,g_fantasy
...,...,...
9738,m_193583,g_fantasy
9739,m_193585,g_drama
9740,m_193587,g_action
9740,m_193587,g_animation


In [6]:
users = ratings['userId'].unique()
movies = ratings['movieId'].unique()
genres = movies_to_genres['genre'].unique()

## Część 2. - spacer po grafie

In [8]:
import collections

# generujemy losowe sciezki w grafie
#   krawedzie reprezentowane sa w dwoch macierzach - ratings i movies
#   w wersji podstawowej wszystkie krawedzie traktujemy jako niewazone i nieskierowane
#   mozliwe ulepszenia:
#    - rozwazenie krawedzi skierowanych
#    - uwzglednienie wag krawedzi (ocen uzytkownikow)
#    - jakas forma normalizacji - obnizenia wag wierzcholkow o wysokich stopniach
#    - Node2Vec - parametry P i Q
# wynikiem powinna byc lista list - kazda z tych list zawiera kolejne ID wierzcholkow na sciezce

# def generate_walks(ratings, movies_to_genres, paths_per_node, path_length):
#     paths = []
#     # ...
#     return paths
    
# walks = generate_walks(ratings, movies_to_genres, PATHS_COUNT_PER_NODE, PATH_LENGTH)


def generate_walks(ratings, movies_to_genres, paths_per_node, path_length):
    edges = get_edge_list(ratings, movies_to_genres)
    paths = []
    
    for user in users:
        paths.append(generate_path(user, edges, path_length))
    for movie in movies:
        paths.append(generate_path(movie, edges, path_length))    
    for genre in genres:
        paths.append(generate_path(genre, edges, path_length))
        
    return paths
    

def get_edge_list(ratings, movies_to_genres):
    edges = collections.defaultdict(lambda: [])
    for index, row in ratings.iterrows():
        edges[row['userId']].append(row['movieId'])
        edges[row['movieId']].append(row['userId'])
    for index, row in movies_to_genres.iterrows():
        edges[row['movieId']].append(row['genre'])
        edges[row['genre']].append(row['movieId'])
    return edges
        
    
def generate_path(start_node_id, edges, path_length):  # node type is user, movie or genre
    path = [start_node_id]
    current = start_node_id
    for i in range(path_length):
        valid_edges = edges[curr_node]
        chosen_next = valid_edges[random.randint(0, len(valid_edges)-1)]
        path.append(chosen_next)
        current = chosen_next
    return path
    
    
    
walks = generate_walks(ratings, movies_to_genres, PATHS_COUNT_PER_NODE, PATH_LENGTH)
edges = get_edge_list(ratings, movies_to_genres)

## Część 3. - obliczenie osadzeń

In [9]:
# trenujemy model
#   zauwaz, ze wszystkie trzy rodzaje wierzcholkow beda reprezentowane tak samo, w tej samej przestrzeni

model = Word2Vec(sentences=walks, size=VECTOR_SIZE, window=NEIGHBOURS_WINDOW, min_count=1, workers=4)
embeddings = model.wv

## Część 4. - rekomendacje i zastosowania

In [10]:
PULP_FICTION = 'm_296'
TOY_STORY = 'm_1'
PLANET_OF_THE_APES = 'm_2529'

In [11]:
# wyszukajmy K najpodobniejszych filmów do danego
# porownaj wyniki dla odleglosci euklidesowej i cosinuswej, np. na trzech powyzszych filmach

def euclidian_distance(i, j):
    return np.linalg.norm(i - j)

def cosine_distance(i, j):
    return np.dot(i, j)/(np.linalg.norm(i) * np.linalg.norm(j))

def k_most_similar_movies(movie_id, K, embeddings, distance_fun):

    similarities = {}
    target = embeddings[movie_id]
    
    for movie in movies:
        similarities[movie] = distance_fun(target, embeddings[movie])
    
    sorted_similarities = sorted(similarities.items(), key=lambda pair: pair[1], reverse=True)
    return sorted_similarities[0:K]

k_most_similar_movies(PULP_FICTION, 5, embeddings, cosine_distance)

[('m_296', 1.0),
 ('m_356', 0.998856),
 ('m_318', 0.99854875),
 ('m_260', 0.998522),
 ('m_110', 0.99848914)]

In [69]:
# sprobujmy czegos bardziej skomplikowanego
#   znajdz ulubiony gatunek filmowy uzytkownika
#   a nastepnie zaproponuj K filmow z tego gatunku - ale nie tych najblizszych uzytkownikowi
#   (zaproponuj, w jaki sposob dobrac filmy interesujace, ale nie z najblizszego otoczenia)

def k_from_favourite_genre(user_id, K, embeddings, distance_fun, gap):
    
    # find fav genre
    
    similarities = {}
    target = embeddings[user_id]
    
    for genre in genres:
        similarities[genre] = distance_fun(target, embeddings[genre])
    
    sorted_similarities = sorted(similarities.items(), key=lambda pair: pair[1], reverse=True)
    favourite_genre =  sorted_similarities[0][0]
    
    # k from fav genre
    
    movies_in_genre = edges[genre]

    similarities = {}
    target = embeddings[genre]
    
    for movie in movies:
        similarities[movie] = distance_fun(target, embeddings[movie])
        
    sorted_similarities = sorted(similarities.items(), key=lambda pair: pair[1], reverse=True)
    return sorted_similarities[gap:gap + K]


gap = 500
k_from_favourite_genre(users[0], 5, embeddings, cosine_distance, gap)


[('m_35836', 0.95047367),
 ('m_103249', 0.9504245),
 ('m_33166', 0.95021796),
 ('m_2424', 0.9501828),
 ('m_96588', 0.95015)]

In [53]:
# Na koniec najbardziej skomplikowany algorytm - odpowiednik "Radia utworu" w Spotify.
#   Zaczynamy od jednego filmu, a nastepnie wyznaczamy kolejne, wedrujac po przestrzeni, w ktorej wszystkie elementy sa osadzone.
#   Zaproponuj, jak zdefiniowac podzbior filmow, z ktorych bedziemy wybierac (np. filmy odlegle o min. a i max. b od danego)
#   oraz jak generowac kolejny skok (tak, zeby seria rekomendacji nie byla zbyt monotonna, ale rownoczesnie zgodna z gustem uzytkownika)

import random
    
def get_playlist(start_movie_id, user_id, K, embeddings, distance_fun):
    
    user = embeddings[user_id]
    target_similarity = distance_fun(user, embeddings[start_movie_id])
    minimum = 0.3
    maximum = 0.7
    
    # radio based on start movie
    
    target_movie = embeddings[start_movie_id]
    radio = []
    
    for movie in movies:
        distance = distance_fun(target_movie, embeddings[movie])
        if minimum <= distance <= maximum:
            radio.append(movie)
    
    # improved radio filtered by user's preferences
    
    better_radio = []
    for movie in radio:
        similarity = distance_fun(user, embeddings[movie])
        if similarity > target_similarity * 0.7:
            better_radio.append((movie, similarity))
            
    playlist = []
    
    for i in range(10):
        next_step = random.randint(0, len(better_radio))
        playlist.append(better_radio[next_step])
    
    if len(playlist) < K:
        return playlist
    else:
        return playlist[0:K]

get_playlist(movies[8], users[10], 10, embeddings, cosine_distance)

[('m_3626', 0.53897834),
 ('m_2092', 0.7677976),
 ('m_2857', 0.65320367),
 ('m_271', 0.8216514),
 ('m_555', 0.9887689),
 ('m_910', 0.9773311),
 ('m_134393', 0.9544149),
 ('m_72142', 0.84791636),
 ('m_353', 0.8977042),
 ('m_575', 0.83878714)]